In [1]:
import numpy as np
import pandas as pd
from steady_BEM import steady_BEM


In [ ]:
# define flow conditions
Uinf = 10 # unperturbed wind speed in m/s
TSR = 10 # tip speed ratio
Radius = 50
N_blade_sec = 30
NBlades = 3

TipLocation_R =  1
RootLocation_R =  0.2
pitch = 2

airfoil = 'DU_polar.txt'

B = steady_BEM(airfoil, TipLocation_R, RootLocation_R, NBlades, Radius, Uinf, TSR, N_blade_sec)
CT,CP, results = B.get_solution(pitch)
print('CT: {}'.format(CT))
print('CP: {}'.format(CP))
    

we are at 0 iterations
we are at 20 iterations
we are at 40 iterations
we are at 60 iterations
we are at 80 iterations
we are at 100 iterations
we are at 120 iterations
we are at 140 iterations
we are at 160 iterations
we are at 180 iterations


In [3]:
B.find_pitch(CT=0.5)

NameError: name 'B' is not defined

In [20]:
B.__dict__

{'airfoil': 'DU_polar.txt',
 'NBlades': 3,
 'Radius': 50,
 'Uinf': 10,
 'Omega': 2.0,
 'N_blade_sec': 30,
 'RootLocation_R': 0.2,
 'TipLocation_R': 1,
 'r_R_dist': array([0.2       , 0.20219124, 0.20874096, 0.21957739, 0.23458182,
        0.25358984, 0.2763932 , 0.30274207, 0.33234776, 0.3648859 ,
        0.4       , 0.43730534, 0.4763932 , 0.51683532, 0.55818861,
        0.6       , 0.64181139, 0.68316468, 0.7236068 , 0.76269466,
        0.8       , 0.8351141 , 0.86765224, 0.89725793, 0.9236068 ,
        0.94641016, 0.96541818, 0.98042261, 0.99125904, 0.99780876,
        1.        ]),
 'r_R_cent': array([0.20109562, 0.2054661 , 0.21415918, 0.22707961, 0.24408583,
        0.26499152, 0.28956764, 0.31754491, 0.34861683, 0.38244295,
        0.41865267, 0.45684927, 0.49661426, 0.53751197, 0.57909431,
        0.62090569, 0.66248803, 0.70338574, 0.74315073, 0.78134733,
        0.81755705, 0.85138317, 0.88245509, 0.91043236, 0.93500848,
        0.95591417, 0.97292039, 0.98584082, 0.9945339 ,